In [1]:
import numpy as np
import pandas as pd
from copy import deepcopy
from IPython.display import display,Latex,Math


In [2]:
data = pd.read_csv("cytometry-1.txt", delimiter=" ").values
cov_mat = np.cov(data.T)
adj_mat = np.zeros((11, 11))
one_ele = [(0, 1), (1, 3), (1, 6), (1, 7), (1, 9), (2, 3), (2, 7), (2, 9), (3, 6), (3, 7), (3, 9), (3, 10),
           (6, 9), (7, 9), (8, 9), (9, 10)]
for pair in one_ele:
    adj_mat[pair[0], pair[1]] = 1
    adj_mat[pair[1], pair[0]] = 1
for i in range(data.shape[1]):
    adj_mat[i, i] = 1
p = list(range(11))
w = cov_mat

In [3]:
def recover_beta(beta_star, no_edge, p_use, n):
    p = len(no_edge) + len(p_use)
    beta = np.zeros(p)
    for i in range(len(p_use)):
        if p_use[i] < n:
            beta[p_use[i]] = beta_star[i]
        else:
            beta[p_use[i] - 1] = beta_star[i]
    return beta

In [4]:
def one_round_iter(data, w, S, p):
    w12_res = []
    beta_res = []
    w_res = np.zeros_like(w)
    for i in range(data.shape[1]):
        w_res[i, i] = S[i, i]
        p_use = deepcopy(p)
        p_use.remove(i)
        w11 = w[p_use]
        w11 = w11[:, p_use]
        no_edge = []
        for j in range(data.shape[1]):
            if adj_mat[i, j] == 0:
                no_edge.append(j)
        for num in no_edge:
            p_use.remove(num)
        if len(p_use) == 0:
            beta = np.zeros(data.shape[1] - 1)
        else:
            w11_star = w[p_use]
            w11_star = w11_star[:, p_use]
            s12 = S[i]
            s12_star = s12[p_use]
            if len(p_use) == 1:
                beta_star = np.squeeze((1 / w11_star) * s12_star, 1)
            elif len(p_use) > 1:
                beta_star = np.dot(np.linalg.inv(w11_star), s12_star)
            beta = recover_beta(beta_star, no_edge, p_use, i)
        w12 = np.dot(w11, beta)
        for j in range(len(w12)):
            if j < i:
                w_res[i, j] = w12[j]
            else:
                w_res[i, j + 1] = w12[j]
        w12_res.append(w12)
        beta_res.append(beta)
    return w_res, w12_res, beta_res

In [5]:
def estimation(data, s, p, eps):
    w_former = s
    w_now, w12_res, beta_res = one_round_iter(data, s, s, p)
    change = np.sum(np.abs(w_now - w_former)) / (w_former.shape[0] * w_former.shape[1])
    n = 1
    while change > eps:
        w_former = w_now
        w_now, w12_res, beta_res = one_round_iter(data, w_former, s, p)
        change = np.sum(np.abs(w_now - w_former)) / (w_former.shape[0] * w_former.shape[1])
        n += 1
    theta = np.zeros_like(s)
    for i in range(data.shape[1]):
        theta[i, i] = 1 / (s[2, 2] - np.dot(w12_res[i].T, beta_res[i]))
        theta12 = -theta[i, i] * beta_res[i]
        for j in range(len(theta12)):
            if j < i:
                theta[i, j] = theta12[j]
            else:
                theta[i, j + 1] = theta12[j]
    return theta,w_now

In [15]:
theta,w_now = estimation(data, w, p, 0.1)

In [54]:
%matplotlib inline
from IPython.core.interactiveshell import InteractiveShell
sh = InteractiveShell.instance()
def trans(x):
    l=x.split('.')
    if int(l[0])>0:
        p=len(l[0])-1
        print(l[0][0]+'.'+l[0][1:]+l[1]+'e'+str(p))
    else:
        p=1
        for i in l[1]:
            if i == '0':
                p+=1
            else:
                break
        t=l[1][p-1:]
        if len(t) > 1:
            t=t[0]+'.'+t[1:]
        return t+'e-'+str(p)
def number_to_str(n,cut=6):
    ns=str(n)
    format_='{0:.'+str(cut)+'f}'
    #if 'e' in ns or ('.' in ns and len(ns)>cut+1):
       # return format_.format(n)
    if n == 0:
        return ns
    if "e" not in ns:
        ns = trans(ns)
    if 'e' in ns and "-" not in ns:
        return ns[:3]+ns[-4:]
    if "-" in ns:
        return ns[:5]+ns[-4:]
    return ns
def matrix_to_latex(mat,style='bmatrix'):
    if type(mat)==np.matrixlib.defmatrix.matrix:
        mat=mat.A
    head=r'\begin{'+style+'}'
    tail=r'\end{'+style+'}'
    if len(mat.shape)==1:
        body=r'\\'.join([str(el) for el in mat])
        return head+body+tail
    elif len(mat.shape)==2:
        lines=[]
        for row in mat:
            lines.append('&'.join([number_to_str(el) for el in row])+r'\\')
        s=head+' '.join(lines)+tail
        return s
    return None
sh.display_formatter.formatters['text/latex'].type_printers[np.ndarray]=matrix_to_latex

In [46]:
for i in range(theta.shape[0]):
    for j in range(theta.shape[1]):
        if theta[i,j] == 0:
            theta[i,j] = 0

In [55]:
theta

array([[-3.34979406e-05,  2.17759063e-05,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.37741597e-05, -9.15833491e-06,  0.00000000e+00,
         2.51431825e-08,  0.00000000e+00,  0.00000000e+00,
         1.20650786e-07, -1.15674410e-08,  0.00000000e+00,
         1.61323883e-08,  0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  2.38095990e-04,
        -1.24966719e-04,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  1.53249490e-06,  0.00000000e+00,
        -4.29777497e-06,  0.00000000e+00],
       [ 0.00000000e+00,  2.67255967e-07,  3.30890895e-05,
        -2.12992090e-05,  0.00000000e+00,  0.00000000e+00,
         2.40330958e-06,  5.24294649e-08,  0.00000000e+00,
        -5.60851910e-07,  1.30958817e-06],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  3.30827681e-05,  0.00000000e+00,
  

1.5804e-2


array([[-3.34979406e-05,  2.17759063e-05,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00,  0.00000000e+00],
       [ 1.37741597e-05, -9.15833491e-06,  0.00000000e+00,
         2.51431825e-08,  0.00000000e+00,  0.00000000e+00,
         1.20650786e-07, -1.15674410e-08,  0.00000000e+00,
         1.61323883e-08,  0.00000000e+00],
       [-0.00000000e+00, -0.00000000e+00,  2.38095990e-04,
        -1.24966719e-04, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  1.53249490e-06, -0.00000000e+00,
        -4.29777497e-06, -0.00000000e+00],
       [ 0.00000000e+00,  2.67255967e-07,  3.30890895e-05,
        -2.12992090e-05,  0.00000000e+00,  0.00000000e+00,
         2.40330958e-06,  5.24294649e-08,  0.00000000e+00,
        -5.60851910e-07,  1.30958817e-06],
       [-0.00000000e+00, -0.00000000e+00, -0.00000000e+00,
        -0.00000000e+00,  3.30827681e-05, -0.00000000e+00,
  

array([[ 61270.15222585,  92420.92989074,   9996.16320966,
         17715.52849431,      0.        ,      0.        ,
         10449.46810349, -25273.34884467,   5338.2471179 ,
         29658.04993063,  11237.07657123],
       [ 92420.92989074, 142171.38796534,  15375.23895453,
         27251.84952752,      0.        ,      0.        ,
         16074.44748206, -38878.06677585,   8210.43850298,
         45623.06533778,  17283.1835006 ],
       [  9996.16320966,  15377.12723218,  30227.21667793,
         48205.33732084,      0.        ,      0.        ,
          9414.13543405, -21652.97792175,   5852.10185144,
         32512.90640469,  14645.78212677],
       [ 17715.88757815,  27251.84952752,  48205.33732084,
         89608.91546425,      0.        ,      0.        ,
         16860.92952645, -34171.45188737,   9506.79222304,
         52817.50960645,  24705.64580037],
       [     0.        ,      0.        ,      0.        ,
             0.        ,   1853.14384438,      0.        ,
  

In [18]:
np.linalg.inv(cov_mat)

array([[ 9.26638447e-04, -6.13682629e-04,  1.66627993e-05,
        -1.11128939e-06, -3.04191516e-05, -9.71620812e-05,
         7.00320749e-05, -1.93812822e-06, -7.34240631e-06,
         2.30157235e-06,  8.31313587e-06],
       [-6.13682629e-04,  4.14932552e-04, -1.37968882e-05,
         1.42300558e-06,  2.04454890e-05,  8.24249173e-05,
        -5.63213625e-05,  1.62649946e-06,  8.64984289e-06,
        -2.88999254e-06, -4.69644884e-06],
       [ 1.66627993e-05, -1.37968882e-05,  2.60791808e-04,
        -1.35878788e-04,  8.78887004e-05,  2.17198506e-05,
        -2.07823379e-05,  1.23294287e-06,  2.13008136e-05,
        -3.22957690e-06, -7.47480002e-06],
       [-1.11128939e-06,  1.42300558e-06, -1.35878788e-04,
         8.68511989e-05, -7.05327913e-05,  3.08756446e-06,
        -2.60825170e-06,  9.10665063e-09, -6.25812187e-06,
         2.57297352e-07,  2.85565541e-07],
       [-3.04191516e-05,  2.04454890e-05,  8.78887004e-05,
        -7.05327913e-05,  6.09603230e-04, -9.92291002e-06,
  